[View in Colaboratory](https://colab.research.google.com/github/pydemia/DeepLearning/blob/master/scripts/DeepLearning/Neural%20Networks%20and%20Deep%20Learning/tf_keras_code_flow_test_colab.ipynb)

In [0]:
import os
import math
import datetime as dt
import itertools as it
from glob import glob
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import tensorflow as tf

import keras
from keras import backend as K
from keras import (regularizers, constraints,
                   initializers, activations)
from keras.layers.recurrent import Recurrent
from keras.models import Sequential, Model
from keras.layers import (SimpleRNN, RNN, LSTM, GRU,
                          Input, Dense, Activation, Lambda,
                          Reshape, Flatten, Permute,
                          Embedding, RepeatVector,
                          TimeDistributed, Bidirectional,
                          dot, multiply, concatenate, merge)
from keras.engine import InputSpec
from keras.callbacks import Callback, LambdaCallback
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.utils import multi_gpu_model

import matplotlib.pyplot as plt
%matplotlib inline

/home/pydemia/apps/anaconda3/envs/tf-py36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/pydemia/apps/anaconda3/envs/tf-py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


np.set_printoptions(precision=10)

# Tensorflow

# Keras

In [0]:
def sigmoid(x, derivative=False):
    return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [0]:
def tanh(x):
    return np.sinh(x)/np.cosh(x)

---
## RNN Code Flow

### Forward-Propagation

In [0]:
units = 1
input_dim = 5

if input_dim == 1:
    data_X = np.array([[.3],
                       [.7],
                       [.4]
                      ]).reshape(1, 3, 1).astype('float32')
elif input_dim == 2:
    data_X = np.array([[.3, .8],
                       [.7, .6],
                       [.4, 0.]
                      ]).reshape(1, 3, 2).astype('float32')

else:
    data_X = np.array([[.3, .8, .1, .7, .2],
                       [.7, .6, .2, .4, .5],
                       [.4, 0., .9, .3, .4]
                      ]).reshape(1, 3, 5).astype('float32')

print(data_X.shape)
pprint(data_X)

(1, 3, 5)
array([[[0.3, 0.8, 0.1, 0.7, 0.2],
        [0.7, 0.6, 0.2, 0.4, 0.5],
        [0.4, 0. , 0.9, 0.3, 0.4]]], dtype=float32)


#### `return_sequences=False`

In [0]:
units = units
#init_state = np.zeros((1, units), dtype=np.float32)
#init_state = [K.zeros(init_state.shape, dtype=np.float32, name=None)]

if len(data_X.shape) == 3:
    n_sample, seq_len, input_dim = data_X.shape
else:
    n_sample, seq_len, input_dim, _ = data_X.shape

# Define an input sequence and process it.
_input_layer = Input(shape=(seq_len, input_dim), name='Input_layer')

_rnn_layer = SimpleRNN(units, return_sequences=False,
                       return_state=True,
                       name='RNN_layer')
outputs, states = _rnn_layer(_input_layer)

#_output_layer = Dense(1, activation='softmax', name='Output_layer')
#_output_layer = Activation('softmax', name='Output_layer')
#_outputs = _output_layer(outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model(inputs=_input_layer, outputs=[outputs, states])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

pprint(model.weights)
model.summary()

[<tf.Variable 'RNN_layer/kernel:0' shape=(5, 1) dtype=float32_ref>,
 <tf.Variable 'RNN_layer/recurrent_kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'RNN_layer/bias:0' shape=(1,) dtype=float32_ref>]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 3, 5)              0         
_________________________________________________________________
RNN_layer (SimpleRNN)        [(None, 1), (None, 1)]    7         
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.output_shape

[(None, 1), (None, 1)]

In [0]:
model.input_shape

(None, 3, 5)

* Create Session Error Exception

In [0]:
try:
    model.get_weights()
except:
    pass

In [0]:
kernel, recurrent_kernel, bias = _rnn_layer.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')
print('Kernel_shape:', kernel.shape)
print('Recurrent_Kernel_shape:', recurrent_kernel.shape)
print('Bias_shape:', bias.shape)

[[-0.68241835]
 [-0.49033356]
 [-0.07335067]
 [-0.28062916]
 [ 0.0865953 ]]
[[1.]]
[0.]
Kernel_shape: (5, 1)
Recurrent_Kernel_shape: (1, 1)
Bias_shape: (1,)


In [0]:
_rnn_layer.get_initial_state(data_X)

[<tf.Tensor 'Tile:0' shape=(1, 1) dtype=float32>]

In [0]:
weight = ['h']

my_kernel = (np.arange(input_dim * units* len(weight)) * .1 + .2).reshape(input_dim, units* len(weight))
my_recurrent_kernel = (np.arange(units * units * len(weight)) * .01 - .05).reshape(units, units * len(weight))
my_bias = np.zeros(units * len(weight)).reshape(units * len(weight), )

print(my_kernel, my_recurrent_kernel, my_bias, sep='\n')

_rnn_layer.set_weights([my_kernel, my_recurrent_kernel, my_bias])

[[0.2]
 [0.3]
 [0.4]
 [0.5]
 [0.6]]
[[-0.05]]
[0.]


In [0]:
kernel, recurrent_kernel, bias = model.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')

return_sequence = False
return_state = True


sequence = []
for _ in range(len(data_X[0])):
    
    inputs = data_X[0][_]
    inputs = inputs.astype(np.float32)
    print('inputs')
    print(inputs.shape)
    pprint(inputs)

    print('input ---------')
    wi = np.dot(inputs, kernel) + bias
    wi = wi.astype(np.float32)
    print(wi.shape)
    pprint(wi)
    #print('\nh:', h)

    if _ == 0:
        initial_state = [np.zeros(units, dtype=np.float32)]
        states = initial_state
    else:
        states = [h]
    prev_h = states[0]
    print('prev h --------')
    print(prev_h.shape)
    pprint(prev_h)
    #print('\nprev:', prev_output)

    #print('w. prev h -----')
    wh = np.dot(prev_h, recurrent_kernel)
    wh = wh.astype(np.float32)
    print(wh.shape)
    pprint(wh)

    print('h --------')
    h = tanh(wh + wi)
    h = h.astype(np.float32)
    print(h.shape)
    pprint(h)

    print('output ---')
    output = h
    output = output.astype(np.float32)
    print(output.shape)
    pprint(output)
    
    
    print('\nstep (%s)' % _ + '='*37)
    print('input\t\t:', inputs)
    print('prev h\t\t:', prev_h)
    print('h\t\t:', h)
    print('output(%s)\t: %s' % (_, output))
    print('='*45 + '\n')
    
    sequence.append(output)
    
    if return_sequence:
        result = sequence
    else:
        result = sequence[-1]
    
    if return_state:
        state = output
    else:
        state = []
        
        
result = np.expand_dims(np.stack(result), axis=0)
state = np.expand_dims(np.array(state), axis=0)

print('\nResult: [Output, State]')
print('\n=== Numpy ===')
print([result, state])
print('\n=== Keras ===')
print(model.predict(data_X))

[[0.2]
 [0.3]
 [0.4]
 [0.5]
 [0.6]]
[[-0.05]]
[0.]
inputs
(5,)
array([0.3, 0.8, 0.1, 0.7, 0.2], dtype=float32)
input ---------
(1,)
array([0.81], dtype=float32)
prev h --------
(1,)
array([0.], dtype=float32)
(1,)
array([-0.], dtype=float32)
h --------
(1,)
array([0.66959035], dtype=float32)
output ---
(1,)
array([0.66959035], dtype=float32)

step (0)=====================================
input		: [0.3 0.8 0.1 0.7 0.2]
prev h		: [0.]
h		: [0.66959035]
output(0)	: [0.66959035]

inputs
(5,)
array([0.7, 0.6, 0.2, 0.4, 0.5], dtype=float32)
input ---------
(1,)
array([0.90000004], dtype=float32)
prev h --------
(1,)
array([0.66959035], dtype=float32)
(1,)
array([-0.03347952], dtype=float32)
h --------
(1,)
array([0.699602], dtype=float32)
output ---
(1,)
array([0.699602], dtype=float32)

step (1)=====================================
input		: [0.7 0.6 0.2 0.4 0.5]
prev h		: [0.66959035]
h		: [0.699602]
output(1)	: [0.699602]

inputs
(5,)
array([0.4, 0. , 0.9, 0.3, 0.4], dtype=float32)
input -

#### `return_sequences=True`

In [0]:
units = units
#units = 3
n_sample, seq_len, input_dim = data_X.shape

# Define an input sequence and process it.
_input_layer = Input(shape=(seq_len, input_dim), name='Input_layer')

_rnn_layer = SimpleRNN(units, return_sequences=True, return_state=True, name='RNN_layer')
outputs, states = _rnn_layer(_input_layer)

#_output_layer = Dense(1, activation='softmax', name='Output_layer')
_output_layer = Activation('softmax', name='Output_layer')
_outputs = _output_layer(outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
#model = Model(inputs=_input_layer, outputs=[outputs, states])
model = Model(inputs=_input_layer, outputs=[_outputs, states])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

pprint(model.weights)
model.summary()

[<tf.Variable 'RNN_layer_1/kernel:0' shape=(5, 1) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_1/recurrent_kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_1/bias:0' shape=(1,) dtype=float32_ref>]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 3, 5)              0         
_________________________________________________________________
RNN_layer (SimpleRNN)        [(None, 3, 1), (None, 1)] 7         
_________________________________________________________________
Output_layer (Activation)    (None, 3, 1)              0         
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.output_shape

[(None, 3, 1), (None, 1)]

In [0]:
model.input_shape

(None, 3, 5)

In [0]:
weight = ['h']

my_kernel = (np.arange(input_dim * units* len(weight)) * .1 + .2).reshape(input_dim, units* len(weight)).astype(np.float32)
my_recurrent_kernel = (np.arange(units * units * len(weight)) * .01 - .05).reshape(units, units * len(weight)).astype(np.float32)
my_bias = np.zeros(units * len(weight)).reshape(units * len(weight), ).astype(np.float32)

print(my_kernel, my_recurrent_kernel, my_bias, sep='\n')

_rnn_layer.set_weights([my_kernel, my_recurrent_kernel, my_bias])

[[0.2]
 [0.3]
 [0.4]
 [0.5]
 [0.6]]
[[-0.05]]
[0.]


In [0]:
kernel, recurrent_kernel, bias = model.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')

return_sequence = True
return_state = True


sequence = []
for _ in range(len(data_X[0])):
    
    inputs = data_X[0][_]
    inputs = inputs.astype(np.float32)
    print('inputs')
    print(inputs.shape)
    pprint(inputs)

    print('input ---------')
    wi = np.dot(inputs, kernel) + bias
    wi = wi.astype(np.float32)
    print(wi.shape)
    pprint(wi)
    #print('\nh:', h)

    if _ == 0:
        initial_state = [np.zeros(units, dtype=np.float32)]
        states = initial_state
    else:
        states = [h]
    prev_h = states[0]
    print('prev h --------')
    print(prev_h.shape)
    pprint(prev_h)
    #print('\nprev:', prev_output)

    #print('w. prev h -----')
    wh = np.dot(prev_h, recurrent_kernel)
    wh = wh.astype(np.float32)
    print(wh.shape)
    pprint(wh)

    print('h --------')
    h = tanh(wh + wi)
    h = h.astype(np.float32)
    print(h.shape)
    pprint(h)

    print('output ---')
    output = h
    output = output.astype(np.float32)
    print(output.shape)
    pprint(output)

    print('\nstep (%s)' % _ + '='*37)
    print('input\t\t:', inputs)
    print('prev h\t\t:', prev_h)
    print('h\t\t:', h)
    print('output(%s)\t: %s' % (_, output))
    print('='*45 + '\n')
    
    sequence.append(output)
    
    if return_sequence:
        result = sequence
    else:
        result = sequence[-1]
    
    if return_state:
        state = output
    else:
        state = []
        
        
result = np.expand_dims(np.stack(result), axis=0)
state = np.expand_dims(np.array(state), axis=0)

print('\nResult: [Output, State]')
print('\n=== Numpy ===')
print([result, state])
print('\n=== Keras ===')
print(model.predict(data_X))

[[0.2]
 [0.3]
 [0.4]
 [0.5]
 [0.6]]
[[-0.05]]
[0.]
inputs
(5,)
array([0.3, 0.8, 0.1, 0.7, 0.2], dtype=float32)
input ---------
(1,)
array([0.81], dtype=float32)
prev h --------
(1,)
array([0.], dtype=float32)
(1,)
array([-0.], dtype=float32)
h --------
(1,)
array([0.66959035], dtype=float32)
output ---
(1,)
array([0.66959035], dtype=float32)

step (0)=====================================
input		: [0.3 0.8 0.1 0.7 0.2]
prev h		: [0.]
h		: [0.66959035]
output(0)	: [0.66959035]

inputs
(5,)
array([0.7, 0.6, 0.2, 0.4, 0.5], dtype=float32)
input ---------
(1,)
array([0.90000004], dtype=float32)
prev h --------
(1,)
array([0.66959035], dtype=float32)
(1,)
array([-0.03347952], dtype=float32)
h --------
(1,)
array([0.699602], dtype=float32)
output ---
(1,)
array([0.699602], dtype=float32)

step (1)=====================================
input		: [0.7 0.6 0.2 0.4 0.5]
prev h		: [0.66959035]
h		: [0.699602]
output(1)	: [0.699602]

inputs
(5,)
array([0.4, 0. , 0.9, 0.3, 0.4], dtype=float32)
input -

### Back-Propagation

In [0]:
from scipy.integrate import odeint
from scipy.misc import derivative

In [0]:
x = .2
f = lambda x: -.3*x + .1
g = lambda x: .4*x - .4

y = g(x)
z = f(g(x))

print('X: %s, Y: %s, Z: %s' % (x, y, z))

X: 0.2, Y: -0.32, Z: 0.196


In [0]:
derivative(f, x)

-0.3

In [0]:
x = 5
f = lambda x: 3*(x**2) - 1

y = f(x)

print('X: %s, Y: %s' % (x, y))

X: 5, Y: 74


In [0]:
derivative(f, x)

30.0

In [0]:
def sigmo(x):
    return 1 / (1 + np.exp(-x))

In [0]:
odeint(f, y, .2)

array([[74.]])

---
## LSTM Code Flow

In [0]:
units = 1
input_dim = 1

if input_dim == 1:
    data_X = np.array([[.2],
                       [.5],
                       [.4]
                      ]).reshape(1, 3, 1).astype('float32')
else:
    data_X = np.array([[.2, .1],
                       [.5, .3],
                       [.4, 0.]
                      ]).reshape(1, 3, 2).astype('float32')

print(data_X.shape)
pprint(data_X)

(1, 3, 1)
array([[[0.2],
        [0.5],
        [0.4]]], dtype=float32)


#### `return_sequences=False`

In [0]:
units = units
n_sample, seq_len, input_dim = data_X.shape

# Define an input sequence and process it.
_input_layer = Input(shape=(seq_len, input_dim), name='Input_layer')

_rnn_layer = LSTM(units, return_sequences=False, return_state=True, name='RNN_layer')
outputs, state_h, state_c = _rnn_layer(_input_layer)

#_output_layer = Dense(1, activation='softmax', name='Output_layer')
#_output_layer = Activation('softmax', name='Output_layer')
#_outputs = _output_layer(outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model(inputs=_input_layer, outputs=[outputs, state_h, state_c])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

pprint(model.weights)
model.summary()

[<tf.Variable 'RNN_layer_2/kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_2/recurrent_kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_2/bias:0' shape=(4,) dtype=float32_ref>]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 3, 1)              0         
_________________________________________________________________
RNN_layer (LSTM)             [(None, 1), (None, 1), (N 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [0]:
kernel, recurrent_kernel, bias = _rnn_layer.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')
print('Kernel_shape:', kernel.shape)
print('Recurrent_Kernel_shape:', recurrent_kernel.shape)
print('Bias_shape:', bias.shape)

[[ 0.00122416  0.15380776  0.8817742  -0.2496478 ]]
[[-0.67465925 -0.6976076   0.23320703 -0.06158761]]
[0. 1. 0. 0.]
Kernel_shape: (1, 4)
Recurrent_Kernel_shape: (1, 4)
Bias_shape: (4,)


In [0]:
weight = ['i', 'f', 'c', 'o']

my_kernel = (np.arange(input_dim * units* len(weight)) * .1 + .2).reshape(input_dim, units* len(weight)).astype(np.float32)
my_recurrent_kernel = (np.arange(units * units * len(weight)) * .01 - .05).reshape(units, units * len(weight)).astype(np.float32)
my_bias = np.zeros(units * len(weight)).reshape(units * len(weight), ).astype(np.float32)

print(my_kernel, my_recurrent_kernel, my_bias, sep='\n')

_rnn_layer.set_weights([my_kernel, my_recurrent_kernel, my_bias])

[[0.2 0.3 0.4 0.5]]
[[-0.05 -0.04 -0.03 -0.02]]
[0. 0. 0. 0.]


In [0]:
kernel, recurrent_kernel, bias = model.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')

return_sequence = False
return_state = True

"""
activation = np.tanh
recurrent_activation = sigmoid
"""

kernel_i = kernel[:,           :units * 1]
kernel_f = kernel[:, units * 1: units * 2]
kernel_c = kernel[:, units * 2: units * 3]
kernel_o = kernel[:, units * 3:]

recurrent_kernel_i = recurrent_kernel[:,          : units * 1]
recurrent_kernel_f = recurrent_kernel[:, units * 1: units * 2]
recurrent_kernel_c = recurrent_kernel[:, units * 2: units * 3]
recurrent_kernel_o = recurrent_kernel[:, units * 3:]

bias_i = bias[         : units * 1]
bias_f = bias[units * 1: units * 2]
bias_c = bias[units * 2: units * 3]
bias_o = bias[units * 3:          ]

sequence = []
for _ in range(len(data_X[0])):
    
    inputs = data_X[0][_]
    inputs = inputs.astype(np.float32)
    print('inputs')
    print(inputs.shape)
    pprint(inputs)

    print('x -----------')
    print('dropout(input) time')
    x_i = np.dot(inputs, kernel_i) + bias_i
    x_f = np.dot(inputs, kernel_f) + bias_f
    x_c = np.dot(inputs, kernel_c) + bias_c
    x_o = np.dot(inputs, kernel_o) + bias_o

    if _ == 0:
        initial_state = [np.zeros(units, dtype=np.float32)] * 2
        states = initial_state
    else:
        states = [h, c]
    h_tm1 = prev_h = states[0]
    c_tm1 = prev_c = states[1]
    print('prev --------')
    print(prev_h.shape, prev_c.shape)
    pprint((prev_h, prev_c))
    #print('\nprev:', prev_output)
    
    print('gate --------')
    print('recurrent dropout(h_tm1) time')
    i = sigmoid(x_i + np.dot(h_tm1, recurrent_kernel_i))
    f = sigmoid(x_f + np.dot(h_tm1, recurrent_kernel_f))
    c = f * c_tm1 +\
        i * sigmoid(x_c + np.dot(h_tm1, recurrent_kernel_c))
    o = np.tanh(x_o + np.dot(h_tm1, recurrent_kernel_o))
    
    h = o * np.tanh(c)
    h = h.astype(np.float32)
    print(h.shape)
    pprint(h)
    #print('\nh:', h)

    print('output ------')
    output = h
    output = output.astype(np.float32)
    print(output.shape)
    pprint(output)
    
    print('\nstep (%s)' % _ + '='*37)
    print('input\t\t:', inputs)
    print('prev h, prev_c\t:', prev_h, prev_c)
    print('h, c\t\t:', h, c)
    print('output(%s)\t: %s' % (_, output))
    print('='*45 + '\n')

    sequence.append(output)
    
    if return_sequence:
        result = sequence
    else:
        result = sequence[-1]
    
    if return_state:
        state = [h, c]
    else:
        state = []
        
        
result = np.expand_dims(np.stack(result), axis=0)
state = np.expand_dims(np.array(state), axis=0)

print('\nResult: [Output, State]')
print('\n=== Numpy ===')
print([result, state])
print('\n=== Keras ===')
print(model.predict(data_X))

[[0.2 0.3 0.4 0.5]]
[[-0.05 -0.04 -0.03 -0.02]]
[0. 0. 0. 0.]
inputs
(1,)
array([0.2], dtype=float32)
x -----------
dropout(input) time
prev --------
(1,) (1,)
(array([0.], dtype=float32), array([0.], dtype=float32))
gate --------
recurrent dropout(h_tm1) time
(1,)
array([0.02582867], dtype=float32)
output ------
(1,)
array([0.02582867], dtype=float32)

step (0)=====================================
input		: [0.2]
prev h, prev_c	: [0.] [0.]
h, c		: [0.02582867] [0.26519388]
output(0)	: [0.02582867]

inputs
(1,)
array([0.5], dtype=float32)
x -----------
dropout(input) time
prev --------
(1,) (1,)
(array([0.02582867], dtype=float32), array([0.26519388], dtype=float32))
gate --------
recurrent dropout(h_tm1) time
(1,)
array([0.09924313], dtype=float32)
output ------
(1,)
array([0.09924313], dtype=float32)

step (1)=====================================
input		: [0.5]
prev h, prev_c	: [0.02582867] [0.26519388]
h, c		: [0.09924313] [0.4308287]
output(1)	: [0.09924313]

inputs
(1,)
array([0.4]

#### `return_sequences=True`

In [0]:
units = units
n_sample, seq_len, input_dim = data_X.shape

# Define an input sequence and process it.
_input_layer = Input(shape=(seq_len, input_dim), name='Input_layer')

_rnn_layer = LSTM(units, return_sequences=True, return_state=True, name='RNN_layer')
outputs, state_h, state_c = _rnn_layer(_input_layer)

#_output_layer = Dense(1, activation='softmax', name='Output_layer')
#_output_layer = Activation('softmax', name='Output_layer')
#_outputs = _output_layer(outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model(inputs=_input_layer, outputs=[outputs, state_h, state_c])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

pprint(model.weights)
model.summary()

[<tf.Variable 'RNN_layer_3/kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_3/recurrent_kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_3/bias:0' shape=(4,) dtype=float32_ref>]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 3, 1)              0         
_________________________________________________________________
RNN_layer (LSTM)             [(None, 3, 1), (None, 1), 12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [0]:
weight = ['i', 'f', 'c', 'o']

my_kernel = (np.arange(input_dim * units* len(weight)) * .1 + .2).reshape(input_dim, units* len(weight)).astype(np.float32)
my_recurrent_kernel = (np.arange(units * units * len(weight)) * .01 - .05).reshape(units, units * len(weight)).astype(np.float32)
my_bias = np.zeros(units * len(weight)).reshape(units * len(weight), ).astype(np.float32)

print(my_kernel, my_recurrent_kernel, my_bias, sep='\n')

_rnn_layer.set_weights([my_kernel, my_recurrent_kernel, my_bias])

[[0.2 0.3 0.4 0.5]]
[[-0.05 -0.04 -0.03 -0.02]]
[0. 0. 0. 0.]


In [0]:
kernel, recurrent_kernel, bias = model.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')

return_sequence = True
return_state = True

"""
activation = np.tanh
recurrent_activation = sigmoid
"""

kernel_i = kernel[:,           :units * 1]
kernel_f = kernel[:, units * 1: units * 2]
kernel_c = kernel[:, units * 2: units * 3]
kernel_o = kernel[:, units * 3:]

recurrent_kernel_i = recurrent_kernel[:,          : units * 1]
recurrent_kernel_f = recurrent_kernel[:, units * 1: units * 2]
recurrent_kernel_c = recurrent_kernel[:, units * 2: units * 3]
recurrent_kernel_o = recurrent_kernel[:, units * 3:]

bias_i = bias[         : units * 1]
bias_f = bias[units * 1: units * 2]
bias_c = bias[units * 2: units * 3]
bias_o = bias[units * 3:          ]

sequence = []
for _ in range(len(data_X[0])):
    
    inputs = data_X[0][_]
    inputs = inputs.astype(np.float32)
    print('inputs')
    print(inputs.shape)
    pprint(inputs)

    print('x -----------')
    print('dropout(input) time')
    x_i = np.dot(inputs, kernel_i) + bias_i
    x_f = np.dot(inputs, kernel_f) + bias_f
    x_c = np.dot(inputs, kernel_c) + bias_c
    x_o = np.dot(inputs, kernel_o) + bias_o

    if _ == 0:
        initial_state = [np.zeros(units, dtype=np.float32)] * 2
        states = initial_state
    else:
        states = [h, c]
    h_tm1 = prev_h = states[0]
    c_tm1 = prev_c = states[1]
    print('prev --------')
    print(prev_h.shape, prev_c.shape)
    pprint((prev_h, prev_c))
    #print('\nprev:', prev_output)
    
    print('gate --------')
    print('recurrent dropout(h_tm1) time')
    i = sigmoid(x_i + np.dot(h_tm1, recurrent_kernel_i))
    f = sigmoid(x_f + np.dot(h_tm1, recurrent_kernel_f))
    c = f * c_tm1 +\
        i * sigmoid(x_c + np.dot(h_tm1, recurrent_kernel_c))
    o = np.tanh(x_o + np.dot(h_tm1, recurrent_kernel_o))
    
    h = o * np.tanh(c)
    h = h.astype(np.float32)
    print(h.shape)
    pprint(h)
    #print('\nh:', h)

    print('output ------')
    output = h
    output = output.astype(np.float32)
    print(output.shape)
    pprint(output)
    
    print('\nstep (%s)' % _ + '='*37)
    print('input\t\t:', inputs)
    print('prev h, prev_c\t:', prev_h, prev_c)
    print('h, c\t\t:', h, c)
    print('output(%s)\t: %s' % (_, output))
    print('='*45 + '\n')

    sequence.append(output)
    
    if return_sequence:
        result = sequence
    else:
        result = sequence[-1]
    
    if return_state:
        state = [h, c]
    else:
        state = []
        
        
result = np.expand_dims(np.stack(result), axis=0)
state = np.expand_dims(np.array(state), axis=0)

print('\nResult: [Output, State]')
print('\n=== Numpy ===')
print([result, state])
print('\n=== Keras ===')
print(model.predict(data_X))

[[0.2 0.3 0.4 0.5]]
[[-0.05 -0.04 -0.03 -0.02]]
[0. 0. 0. 0.]
inputs
(1,)
array([0.2], dtype=float32)
x -----------
dropout(input) time
prev --------
(1,) (1,)
(array([0.], dtype=float32), array([0.], dtype=float32))
gate --------
recurrent dropout(h_tm1) time
(1,)
array([0.02582867], dtype=float32)
output ------
(1,)
array([0.02582867], dtype=float32)

step (0)=====================================
input		: [0.2]
prev h, prev_c	: [0.] [0.]
h, c		: [0.02582867] [0.26519388]
output(0)	: [0.02582867]

inputs
(1,)
array([0.5], dtype=float32)
x -----------
dropout(input) time
prev --------
(1,) (1,)
(array([0.02582867], dtype=float32), array([0.26519388], dtype=float32))
gate --------
recurrent dropout(h_tm1) time
(1,)
array([0.09924313], dtype=float32)
output ------
(1,)
array([0.09924313], dtype=float32)

step (1)=====================================
input		: [0.5]
prev h, prev_c	: [0.02582867] [0.26519388]
h, c		: [0.09924313] [0.4308287]
output(1)	: [0.09924313]

inputs
(1,)
array([0.4]

---
## GRU Code Flow

In [0]:
units = 2
input_dim = 2

if input_dim == 1:
    data_X = np.array([[.2],
                       [.5],
                       [.4]
                      ]).reshape(1, 3, 1).astype('float32')
else:
    data_X = np.array([[.2, .1],
                       [.5, .3],
                       [.4, 0.]
                      ]).reshape(1, 3, 2).astype('float32')

print(data_X.shape)
pprint(data_X)

(1, 3, 2)
array([[[0.2, 0.1],
        [0.5, 0.3],
        [0.4, 0. ]]], dtype=float32)


#### `return_sequences=False`

In [0]:
units = units
n_sample, seq_len, input_dim = data_X.shape

# Define an input sequence and process it.
_input_layer = Input(shape=(seq_len, input_dim), name='Input_layer')

_rnn_layer = GRU(units, return_sequences=False, return_state=True, name='RNN_layer')
outputs, states = _rnn_layer(_input_layer)

#_output_layer = Dense(1, activation='softmax', name='Output_layer')
#_output_layer = Activation('softmax', name='Output_layer')
#_outputs = _output_layer(outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model(inputs=_input_layer, outputs=[outputs, states])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

pprint(model.weights)
model.summary()

[<tf.Variable 'RNN_layer_4/kernel:0' shape=(2, 6) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_4/recurrent_kernel:0' shape=(2, 6) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_4/bias:0' shape=(6,) dtype=float32_ref>]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 3, 2)              0         
_________________________________________________________________
RNN_layer (GRU)              [(None, 2), (None, 2)]    30        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________


In [0]:
kernel, recurrent_kernel, bias = _rnn_layer.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')
print('Kernel_shape:', kernel.shape)
print('Recurrent_Kernel_shape:', recurrent_kernel.shape)
print('Bias_shape:', bias.shape)

[[-0.7015311   0.08663237  0.01490903 -0.5300734   0.16394049 -0.7903858 ]
 [ 0.45310313 -0.06462902  0.7931908   0.18173534 -0.04084522 -0.55416363]]
[[-0.5284298  -0.01888672  0.67646074 -0.2728485   0.05823601  0.43007952]
 [-0.22994931  0.15907124 -0.06359684 -0.27196684 -0.8899605  -0.22755088]]
[0. 0. 0. 0. 0. 0.]
Kernel_shape: (2, 6)
Recurrent_Kernel_shape: (2, 6)
Bias_shape: (6,)


In [0]:
weight = ['z', 'r', 'h']

my_kernel = (np.arange(input_dim * units* len(weight)) * .1 + .2).reshape(input_dim, units* len(weight)).astype(np.float32)
my_recurrent_kernel = (np.arange(units * units * len(weight)) * .01 - .05).reshape(units, units * len(weight)).astype(np.float32)
my_bias = np.zeros(units * len(weight)).reshape(units * len(weight), ).astype(np.float32)

print(my_kernel, my_recurrent_kernel, my_bias, sep='\n')

_rnn_layer.set_weights([my_kernel, my_recurrent_kernel, my_bias])

[[0.2 0.3 0.4 0.5 0.6 0.7]
 [0.8 0.9 1.  1.1 1.2 1.3]]
[[-0.05 -0.04 -0.03 -0.02 -0.01  0.  ]
 [ 0.01  0.02  0.03  0.04  0.05  0.06]]
[0. 0. 0. 0. 0. 0.]


In [0]:
kernel, recurrent_kernel, bias = model.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')

return_sequence = False
return_state = True

"""
activation = np.tanh
recurrent_activation = sigmoid
"""

kernel_z = kernel[:,          : units * 1]
kernel_r = kernel[:, units * 1: units * 2]
kernel_h = kernel[:, units * 2:]

recurrent_kernel_z = recurrent_kernel[:,          : units * 1]
recurrent_kernel_r = recurrent_kernel[:, units * 1: units * 2]
recurrent_kernel_h = recurrent_kernel[:, units * 2:]

bias_z = bias[         : units * 1]
bias_r = bias[units    : units * 2]
bias_h = bias[units * 2:]

sequence = []
for _ in range(len(data_X[0])):
    
    inputs = data_X[0][_]
    inputs = inputs.astype(np.float32)
    print('inputs')
    print(inputs.shape)
    pprint(inputs)

    print('x -----------')
    print('dropout(input) time')
    x_z = np.dot(inputs, kernel_z) + bias_z
    x_r = np.dot(inputs, kernel_r) + bias_r
    x_h = np.dot(inputs, kernel_h) + bias_h

    if _ == 0:
        initial_state = [np.zeros(units, dtype=np.float32)]
        states = initial_state
    else:
        states = [h]
    h_tm1 = prev_h = states[0]
    print('prev --------')
    print(prev_h.shape)
    pprint(prev_h)
    #print('\nprev:', prev_output)
    
    print('gate --------')
    print('recurrent dropout(h_tm1) time')
    z = sigmoid(x_z + np.dot(h_tm1, recurrent_kernel_z))
    r = sigmoid(x_r + np.dot(h_tm1, recurrent_kernel_r))
    hh = np.tanh(x_h + np.dot(h_tm1, recurrent_kernel_h))
    
    h = z * h_tm1 + (1 - z) * hh
    h = h.astype(np.float32)
    print(h.shape)
    pprint(h)
    #print('\nh:', h)

    print('output ------')
    output = h
    output = output.astype(np.float32)
    print(output.shape)
    pprint(output)
    
    print('\nstep (%s)' % _ + '='*37)
    print('input\t\t:', inputs)
    print('prev h\t\t:', prev_h)
    print('h\t\t:', h)
    print('output(%s)\t: %s' % (_, output))
    print('='*45 + '\n')
    
    sequence.append(output)
    
    if return_sequence:
        result = sequence
    else:
        result = sequence[-1]
    
    if return_state:
        state = output
    else:
        state = []
        
        
result = np.expand_dims(np.stack(result), axis=0)
state = np.expand_dims(np.array(state), axis=0)

print('\nResult: [Output, State]')
print('\n=== Numpy ===')
print([result, state])
print('\n=== Keras ===')
print(model.predict(data_X))

[[0.2 0.3 0.4 0.5 0.6 0.7]
 [0.8 0.9 1.  1.1 1.2 1.3]]
[[-0.05 -0.04 -0.03 -0.02 -0.01  0.  ]
 [ 0.01  0.02  0.03  0.04  0.05  0.06]]
[0. 0. 0. 0. 0. 0.]
inputs
(2,)
array([0.2, 0.1], dtype=float32)
x -----------
dropout(input) time
prev --------
(2,)
array([0., 0.], dtype=float32)
gate --------
recurrent dropout(h_tm1) time
(2,)
array([0.11069148, 0.12194498], dtype=float32)
output ------
(2,)
array([0.11069148, 0.12194498], dtype=float32)

step (0)=====================================
input		: [0.2 0.1]
prev h		: [0. 0.]
h		: [0.11069148 0.12194498]
output(0)	: [0.11069148 0.12194498]

inputs
(2,)
array([0.5, 0.3], dtype=float32)
x -----------
dropout(input) time
prev --------
(2,)
array([0.11069148, 0.12194498], dtype=float32)
gate --------
recurrent dropout(h_tm1) time
(2,)
array([0.3070247 , 0.32504663], dtype=float32)
output ------
(2,)
array([0.3070247 , 0.32504663], dtype=float32)

step (1)=====================================
input		: [0.5 0.3]
prev h		: [0.11069148 0.12194498

#### `return_sequences=True`

In [0]:
units = units
n_sample, seq_len, input_dim = data_X.shape

# Define an input sequence and process it.
_input_layer = Input(shape=(seq_len, input_dim), name='Input_layer')

_rnn_layer = GRU(units, return_sequences=True, return_state=True, name='RNN_layer')
outputs, states = _rnn_layer(_input_layer)

#_output_layer = Dense(1, activation='softmax', name='Output_layer')
#_output_layer = Activation('softmax', name='Output_layer')
#_outputs = _output_layer(outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model(inputs=_input_layer, outputs=[outputs, states])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

pprint(model.weights)
model.summary()

[<tf.Variable 'RNN_layer_5/kernel:0' shape=(2, 6) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_5/recurrent_kernel:0' shape=(2, 6) dtype=float32_ref>,
 <tf.Variable 'RNN_layer_5/bias:0' shape=(6,) dtype=float32_ref>]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 3, 2)              0         
_________________________________________________________________
RNN_layer (GRU)              [(None, 3, 2), (None, 2)] 30        
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________


In [0]:
weight = ['z', 'r', 'h']

my_kernel = (np.arange(input_dim * units* len(weight)) * .1 + .2).reshape(input_dim, units* len(weight)).astype(np.float32)
my_recurrent_kernel = (np.arange(units * units * len(weight)) * .01 - .05).reshape(units, units * len(weight)).astype(np.float32)
my_bias = np.zeros(units * len(weight)).reshape(units * len(weight), ).astype(np.float32)

print(my_kernel, my_recurrent_kernel, my_bias, sep='\n')

_rnn_layer.set_weights([my_kernel, my_recurrent_kernel, my_bias])

[[0.2 0.3 0.4 0.5 0.6 0.7]
 [0.8 0.9 1.  1.1 1.2 1.3]]
[[-0.05 -0.04 -0.03 -0.02 -0.01  0.  ]
 [ 0.01  0.02  0.03  0.04  0.05  0.06]]
[0. 0. 0. 0. 0. 0.]


In [0]:
kernel, recurrent_kernel, bias = model.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')

return_sequence = True
return_state = True

"""
activation = np.tanh
recurrent_activation = sigmoid
"""

kernel_z = kernel[:,          : units * 1]
kernel_r = kernel[:, units * 1: units * 2]
kernel_h = kernel[:, units * 2:]

recurrent_kernel_z = recurrent_kernel[:,          : units * 1]
recurrent_kernel_r = recurrent_kernel[:, units * 1: units * 2]
recurrent_kernel_h = recurrent_kernel[:, units * 2:]

bias_z = bias[         : units * 1]
bias_r = bias[units    : units * 2]
bias_h = bias[units * 2:]

sequence = []
for _ in range(len(data_X[0])):
    
    inputs = data_X[0][_]
    inputs = inputs.astype(np.float32)
    print('inputs')
    print(inputs.shape)
    pprint(inputs)

    print('x -----------')
    print('dropout(input) time')
    x_z = np.dot(inputs, kernel_z) + bias_z
    x_r = np.dot(inputs, kernel_r) + bias_r
    x_h = np.dot(inputs, kernel_h) + bias_h

    if _ == 0:
        initial_state = [np.zeros(units, dtype=np.float32)]
        states = initial_state
    else:
        states = [h]
    h_tm1 = prev_h = states[0]
    print('prev --------')
    print(prev_h.shape)
    pprint(prev_h)
    #print('\nprev:', prev_output)
    
    print('gate --------')
    print('recurrent dropout(h_tm1) time')
    z = sigmoid(x_z + np.dot(h_tm1, recurrent_kernel_z))
    r = sigmoid(x_r + np.dot(h_tm1, recurrent_kernel_r))
    hh = np.tanh(x_h + np.dot(h_tm1, recurrent_kernel_h))
    
    h = z * h_tm1 + (1 - z) * hh
    h = h.astype(np.float32)
    print(h.shape)
    pprint(h)
    #print('\nh:', h)

    print('output ------')
    output = h
    output = output.astype(np.float32)
    print(output.shape)
    pprint(output)
    
    print('\nstep (%s)' % _ + '='*37)
    print('input\t\t:', inputs)
    print('prev h\t\t:', prev_h)
    print('h\t\t:', h)
    print('output(%s)\t: %s' % (_, output))
    print('='*45 + '\n')
    
    sequence.append(output)
    
    if return_sequence:
        result = sequence
    else:
        result = sequence[-1]
    
    if return_state:
        state = output
    else:
        state = []
        
        
result = np.expand_dims(np.stack(result), axis=0)
state = np.expand_dims(np.array(state), axis=0)

print('\nResult: [Output, State]')
print('\n=== Numpy ===')
print([result, state])
print('\n=== Keras ===')
print(model.predict(data_X))

[[0.2 0.3 0.4 0.5 0.6 0.7]
 [0.8 0.9 1.  1.1 1.2 1.3]]
[[-0.05 -0.04 -0.03 -0.02 -0.01  0.  ]
 [ 0.01  0.02  0.03  0.04  0.05  0.06]]
[0. 0. 0. 0. 0. 0.]
inputs
(2,)
array([0.2, 0.1], dtype=float32)
x -----------
dropout(input) time
prev --------
(2,)
array([0., 0.], dtype=float32)
gate --------
recurrent dropout(h_tm1) time
(2,)
array([0.11069148, 0.12194498], dtype=float32)
output ------
(2,)
array([0.11069148, 0.12194498], dtype=float32)

step (0)=====================================
input		: [0.2 0.1]
prev h		: [0. 0.]
h		: [0.11069148 0.12194498]
output(0)	: [0.11069148 0.12194498]

inputs
(2,)
array([0.5, 0.3], dtype=float32)
x -----------
dropout(input) time
prev --------
(2,)
array([0.11069148, 0.12194498], dtype=float32)
gate --------
recurrent dropout(h_tm1) time
(2,)
array([0.3070247 , 0.32504663], dtype=float32)
output ------
(2,)
array([0.3070247 , 0.32504663], dtype=float32)

step (1)=====================================
input		: [0.5 0.3]
prev h		: [0.11069148 0.12194498

---
## RNN with Word Embedding

In [0]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[23, 17], [44, 3], [6, 10], [5, 3], [15], [40], [49, 10], [13, 44], [49, 3], [14, 48, 17, 46]]
[[23 17  0  0]
 [44  3  0  0]
 [ 6 10  0  0]
 [ 5  3  0  0]
 [15  0  0  0]
 [40  0  0  0]
 [49 10  0  0]
 [13 44  0  0]
 [49  3  0  0]
 [14 48 17 46]]


In [0]:
padded_docs.shape

(10, 4)

In [0]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(SimpleRNN(1, return_sequences=True, return_state=False))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 4, 1)              10        
Total params: 410
Trainable params: 410
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.input_shape

(None, 4)

In [0]:
model.output_shape

(None, 4, 1)

In [0]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

ValueError: Error when checking target: expected simple_rnn_1 to have 3 dimensions, but got array with shape (10, 1)

### Forward-Propagation

In [0]:
units = 2
input_dim = 2

if input_dim == 1:
    data_X = np.array([[.3],
                       [.7],
                       [.4]
                      ]).reshape(1, 3, 1).astype('float32')
else:
    data_X = np.array([[.3, .8],
                       [.7, .6],
                       [.4, 0.]
                      ]).reshape(1, 3, 2).astype('float32')

print(data_X.shape)
pprint(data_X)

#### `return_sequences=False`

In [0]:
units = units
#init_state = np.zeros((1, units), dtype=np.float32)
#init_state = [K.zeros(init_state.shape, dtype=np.float32, name=None)]

if len(data_X.shape) == 3:
    n_sample, seq_len, input_dim = data_X.shape
else:
    n_sample, seq_len, input_dim, _ = data_X.shape

# Define an input sequence and process it.
_input_layer = Input(shape=(seq_len, input_dim), name='Input_layer')

_rnn_layer = SimpleRNN(units, return_sequences=False,
                       return_state=True,
                       name='RNN_layer')
outputs, states = _rnn_layer(_input_layer)

#_output_layer = Dense(1, activation='softmax', name='Output_layer')
#_output_layer = Activation('softmax', name='Output_layer')
#_outputs = _output_layer(outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model(inputs=_input_layer, outputs=[outputs, states])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

pprint(model.weights)
model.summary()

In [0]:
model.output_shape

In [0]:
model.input_shape

* Create Session Error Exception

In [0]:
try:
    model.get_weights()
except:
    pass

In [0]:
kernel, recurrent_kernel, bias = _rnn_layer.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')
print('Kernel_shape:', kernel.shape)
print('Recurrent_Kernel_shape:', recurrent_kernel.shape)
print('Bias_shape:', bias.shape)

In [0]:
_rnn_layer.get_initial_state(data_X)

In [0]:
weight = ['h']

my_kernel = (np.arange(input_dim * units* len(weight)) * .1 + .2).reshape(input_dim, units* len(weight))
my_recurrent_kernel = (np.arange(units * units * len(weight)) * .01 - .05).reshape(units, units * len(weight))
my_bias = np.zeros(units * len(weight)).reshape(units * len(weight), )

print(my_kernel, my_recurrent_kernel, my_bias, sep='\n')

_rnn_layer.set_weights([my_kernel, my_recurrent_kernel, my_bias])

In [0]:
kernel, recurrent_kernel, bias = model.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')

return_sequence = False
return_state = True


sequence = []
for _ in range(len(data_X[0])):
    
    inputs = data_X[0][_]
    inputs = inputs.astype(np.float32)
    print('inputs')
    print(inputs.shape)
    pprint(inputs)

    print('h -----------')
    h = np.dot(inputs, kernel) + bias
    h = h.astype(np.float32)
    print(h.shape)
    pprint(h)
    #print('\nh:', h)

    if _ == 0:
        initial_state = [np.zeros(units, dtype=np.float32)]
        states = initial_state
    else:
        states = [h]
    prev_output = states[0]
    print('prev --------')
    print(prev_output.shape)
    pprint(prev_output)
    #print('\nprev:', prev_output)

    print('a -----------')
    a = np.dot(prev_output, recurrent_kernel)
    a = a.astype(np.float32)
    print(a.shape)
    pprint(a)

    print('output ------')
    output = h + a
    output = output.astype(np.float32)
    print(output.shape)
    pprint(output)
    
    
    print('\nstep (%s)' % _ + '='*37)
    print('h\t\t:', h)
    print('prev\t\t:', prev_output)
    print('output(%s)\t: %s' % (_, output))
    print('='*45 + '\n')
    
    sequence.append(output)
    
    if return_sequence:
        result = sequence
    else:
        result = sequence[-1]
    
    if return_state:
        state = output
    else:
        state = []
        
        
result = np.expand_dims(np.stack(result), axis=0)
state = np.expand_dims(np.array(state), axis=0)

print('\nResult: [Output, State]')
print('\n=== Numpy ===')
print([result, state])
print('\n=== Keras ===')
print(model.predict(data_X))

#### `return_sequences=True`

In [0]:
units = units
n_sample, seq_len, input_dim = data_X.shape

# Define an input sequence and process it.
_input_layer = Input(shape=(seq_len, input_dim), name='Input_layer')

_rnn_layer = SimpleRNN(units, return_sequences=True, return_state=True, name='RNN_layer')
outputs, states = _rnn_layer(_input_layer)

#_output_layer = Dense(1, activation='softmax', name='Output_layer')
#_output_layer = Activation('softmax', name='Output_layer')
#_outputs = _output_layer(outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model(inputs=_input_layer, outputs=[outputs, states])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

pprint(model.weights)
model.summary()

In [0]:
model.output_shape

In [0]:
model.input_shape

In [0]:
weight = ['h']

my_kernel = (np.arange(input_dim * units* len(weight)) * .1 + .2).reshape(input_dim, units* len(weight)).astype(np.float32)
my_recurrent_kernel = (np.arange(units * units * len(weight)) * .01 - .05).reshape(units, units * len(weight)).astype(np.float32)
my_bias = np.zeros(units * len(weight)).reshape(units * len(weight), ).astype(np.float32)

print(my_kernel, my_recurrent_kernel, my_bias, sep='\n')

_rnn_layer.set_weights([my_kernel, my_recurrent_kernel, my_bias])

In [0]:
kernel, recurrent_kernel, bias = model.get_weights()
print(kernel, recurrent_kernel, bias, sep='\n')

return_sequence = True
return_state = True


sequence = []
for _ in range(len(data_X[0])):
    
    inputs = data_X[0][_]
    inputs = inputs.astype(np.float32)
    print('inputs')
    print(inputs.shape)
    pprint(inputs)

    print('h -----------')
    h = np.dot(inputs, kernel) + bias
    h = h.astype(np.float32)
    print(h.shape)
    pprint(h)
    #print('\nh:', h)

    if _ == 0:
        initial_state = [np.zeros(units, dtype=np.float32)]
        states = initial_state
    else:
        states = [output]
    prev_output = states[0]
    print('prev --------')
    print(prev_output.shape)
    pprint(prev_output)
    #print('\nprev:', prev_output)

    print('a -----------')
    a = np.dot(prev_output, recurrent_kernel)
    a = a.astype(np.float32)
    print(a.shape)
    pprint(a)

    print('output ------')
    output = h + a
    output = output.astype(np.float32)
    print(output.shape)
    pprint(output)
    
    
    print('\nstep (%s)' % _ + '='*37)
    print('h\t\t:', h)
    print('prev\t\t:', prev_output)
    print('output(%s)\t: %s' % (_, output))
    print('='*45 + '\n')
    
    sequence.append(output)
    
    if return_sequence:
        result = sequence
    else:
        result = sequence[-1]
    
    if return_state:
        state = output
    else:
        state = []
        
        
result = np.expand_dims(np.stack(result), axis=0)
state = np.expand_dims(np.array(state), axis=0)

print('\nResult: [Output, State]')
print('\n=== Numpy ===')
print([result, state])
print('\n=== Keras ===')
print(model.predict(data_X))

Done.